In [6]:
# !pip install datasets transformers
# !pip install pillow
# !pip install datasets --upgrade (otherwise issues to load the bean dataset)
# !pip install matplotlib

In [ ]:
# Run once, unless you have any update for the data structure
# create files with 47 channels at folder csiseminar/project/dataset/47_channels/
# the output is relevant to check that the files are not only zeros (viewing them, it looks like 
# it is only grey
# %run preprocess_tif_data.ipynb

In [5]:
from transformers.utils.dummy_vision_objects import ImageGPTFeatureExtractor
import random
from PIL import ImageDraw, ImageFont, Image

/Users/eilaarich-landkof-stanford/miniconda3/envs/seminarEnv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [18]:
import os
path_to_non_progressors_images = "/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset/47_channels/labels/nonprogressor/"
path_to_progressors_images = "/Users/eilaarich-landkof-stanford/Documents/Code/csiseminar/project/dataset/47_channels/labels/progressor/"



In [48]:
import os
%run prepare_dataset_from_image_files.ipynb
non_progressors_images_list = os.listdir(path_to_non_progressors_images)
progressors_images_list = os.listdir(path_to_progressors_images)
non_progressors_images_list = [os.path.join(path_to_non_progressors_images,x) for x in non_progressors_images_list]
progressors_images_list = [os.path.join(path_to_progressors_images,x) for x in progressors_images_list]


non_progressors_images_dict = load_47_channel_tif_to_dataset_object(non_progressors_images_list,"non_progressor")
progressors_images_dict = load_47_channel_tif_to_dataset_object(progressors_images_list,"progressor")

Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shape: (47, 1024, 1024)
Image shap

In [49]:
# Create a Dataset object using Dataset.from_dict
dataset_prog = Dataset.from_dict(progressors_images_dict)
dataset_non_prog = Dataset.from_dict(non_progressors_images_dict)

In [50]:
type(dataset_non_prog)

datasets.arrow_dataset.Dataset

In [51]:
from datasets import Dataset, concatenate_datasets

# Concatenate the datasets
ds = concatenate_datasets([dataset_prog, dataset_non_prog])
type(ds)

datasets.arrow_dataset.Dataset

In [55]:
# split to train / validation and test
type(ds)

datasets.arrow_dataset.Dataset

In [65]:
len(ds)

58

In [78]:
%run prepare_dataset_from_image_files.ipynb

train_percentage = 0.6
validation_percentage = 0.2
test_percentage = 0.2

split_datasets = split_custom_dataset(ds, train_percentage, validation_percentage, test_percentage)

# Access individual splits
train_split = split_datasets["train"]
validation_split = split_datasets["validation"]
test_split = split_datasets["test"]

# Print the number of samples in each split
print("Train samples:", len(train_split))
print("Validation samples:", len(validation_split))
print("Test samples:", len(test_split))

Train samples: 34
Validation samples: 11
Test samples: 13


In [79]:
len(train_split['label'])

34

In [85]:
train_split

Dataset({
    features: ['image', 'label'],
    num_rows: 34
})

In [86]:
%run images_methods.ipynb

In [99]:
# # TODO: work on that
# from PIL import Image
# import numpy as np
# from datasets import load_dataset


# # Access an image sample (replace 0 with the desired index)
# sample_index = 0
# image_data = train_split[sample_index]["image"]

# # Convert image data (assumed to be a NumPy array) to a PIL image
# pil_image = Image.fromarray(image_data)

# # Display the PIL image using the default image viewer
# pil_image.show()


In [100]:
# Loading the feature extractor 
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)


/Users/eilaarich-landkof-stanford/miniconda3/envs/seminarEnv/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [101]:
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [ ]:
train_split[0]

In [ ]:
%run prepare_dataset_From_image_files.ipynb
sample_index = 0  # Replace with the desired sample index
sample = get_single_sample(dataset_name, sample_index)

# Print the keys and values of the sample
for key, value in sample.items():
    print(f"{key}: {value}")


In [102]:
feature_extractor(train_split[, return_tensors='pt')

NameError: name 'image' is not defined

In [103]:
def process_example(example):
    inputs = feature_extractor(example['image'], return_tensors='pt')
    inputs['labels'] = example['labels']
    return inputs